데이터 로드

implicit 라이브러리 버전 0.4.0만 쓸 것. 상위 버전에선 변수 할당 오류가 일어나며, 해결할 수 없음 

In [1]:
!pip install implicit==0.4.0

     |████████████████████████████████| 1.1MB 8.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.0-cp36-cp36m-linux_x86_64.whl size=3390316 sha256=03d766e53397adc0c7c9fc238d6f26b089c972fbafdaa219aa348d900c6ca3a2
  Stored in directory: /root/.cache/pip/wheels/fd/d1/66/84e83191d9f626c521271023e4ef83e634e3b8c154f2c4b2da
Successfully built implicit


In [2]:
from implicit.bpr import BayesianPersonalizedRanking as BPR
from implicit.lmf import LogisticMatrixFactorization as LMF
from scipy.sparse import csr_matrix, vstack
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

In [8]:
tr = pd.read_json("train.json")
te = pd.read_json("test.json")

tr_songs = tr.songs.tolist()
te_songs = te.songs.tolist()
tr_tags = tr.tags.tolist()
te_tags = te.tags.tolist()
te_ids = te.id.tolist()

## songs sparse matrix

In [9]:
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_songs):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

n_items = len(iid_to_idx)

idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}

te = []

idx = 0
for i, l in enumerate(te_songs):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)

In [10]:
rows = np.repeat(0,len(tr[0]))
cols = tr[0]
vals = np.repeat(1,len(tr[0]))
tr_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # train 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(tr))): # train 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(tr[i]))
    cols = tr[i]
    vals = np.repeat(1,len(tr[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    tr_csr = vstack([tr_csr, new_csr])


rows = np.repeat(0,len(te[0]))
cols = te[0]
vals = np.repeat(1,len(te[0]))
te_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # test(validation) 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(te))): # test(validation) 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(te[i]))
    cols = te[i]
    vals = np.repeat(1,len(te[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    te_csr = vstack([te_csr, new_csr]) 

r = vstack([te_csr, tr_csr]).tocoo() # test, train 데이터를 병합하고 coo_matrix로 변환

In [12]:
def unique_item(array1, array2): # array1에 있는 원소들 중 array2에도 있는 것들을 제거하는 함수
    array1 = list(array1)
    for x in array1:
        if x in array2:
            array1.remove(x)
    return array1

## Bayesian Personalized Ranking Model for songs

In [ ]:
model = BPR(factors = 400, regularization = 0.2)
model.fit(r.T, show_progress = True)

item_result = []

In [ ]:
for u in tqdm(range(te_csr.shape[0])):
    item_recommend = model.recommend(u, tr_csr, N = 200)
    item_recommend = [x[0] for x in item_recommend]
    item_recommend = unique_item(item_recommend, te[u])
    item_recommend = [idx_to_iid[x] for x in item_recommend][:100]
    item_result.append(item_recommend)

In [ ]:
print(type(item_result), len(item_result))

<class 'list'> 23015
<class 'list'> 23015


In [ ]:
np.array(item_result).shape

(23015, 100)

(23015, 100)

## tags도 똑같이 해본다.


In [ ]:
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_tags):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

n_items = len(iid_to_idx)

idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}

te = []

idx = 0
for i, l in enumerate(te_tags):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)

In [ ]:
rows = np.repeat(0,len(tr[0]))
cols = tr[0]
vals = np.repeat(1,len(tr[0]))
tr_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # train 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(tr))): # train 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(tr[i]))
    cols = tr[i]
    vals = np.repeat(1,len(tr[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    tr_csr = vstack([tr_csr, new_csr])


rows = np.repeat(0,len(te[0]))
cols = te[0]
vals = np.repeat(1,len(te[0]))
te_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # test(validation) 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(te))): # test(validation) 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(te[i]))
    cols = te[i]
    vals = np.repeat(1,len(te[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    te_csr = vstack([te_csr, new_csr]) 

r = vstack([te_csr, tr_csr]).tocoo() # test, train 데이터를 병합하고 coo_matrix로 변환

## Bayesian Personalized Ranking Model for tag

In [ ]:
model = BPR(factors = 256, regularization = 0.01)
model.fit(r.T, show_progress = True)

tag_result = []

In [ ]:
for u in tqdm(range(te_csr.shape[0])):
    tag_recommend = model.recommend(u, tr_csr, N = 200)
    tag_recommend = [x[0] for x in tag_recommend]
    tag_recommend = unique_item(tag_recommend, te[u])
    tag_recommend = [idx_to_iid[x] for x in tag_recommend][:10]
    tag_result.append(tag_recommend)

In [ ]:
print(type(tag_result), len(tag_result))

In [ ]:
np.array(tag_result).shape

(23015, 10)

In [ ]:
returnval = []
for _id, rec, tag_rec in tqdm(zip(te_ids, item_result, tag_result)):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [ ]:
import json
with open('ret.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))

## Logistic Matrix Factorization model for songs

In [13]:

from implicit.lmf import LogisticMatrixFactorization as LMF
model_logit = LMF(factors = 50, regularization = 0.1, iterations = 50)
model_logit.fit(r.T, show_progress = True)

item_result_logit = []

for u in tqdm(range(te_csr.shape[0])):
    item_recommend = model_logit.recommend(u, tr_csr, N = 200)
    item_recommend = [x[0] for x in item_recommend]
    item_recommend = unique_item(item_recommend, te[u])
    item_recommend = [idx_to_iid[x] for x in item_recommend][:100]
    item_result_logit.append(item_recommend)


100%|██████████| 50/50 [06:13<00:00,  7.47s/it]


In [14]:
tr = []
iid_to_idx = {}
tag_to_idx = {} 
idx = 0

for i, l in enumerate(tr_tags):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

n_items = len(iid_to_idx)

idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}

te = []

idx = 0
for i, l in enumerate(te_tags):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)

In [15]:
rows = np.repeat(0,len(tr[0]))
cols = tr[0]
vals = np.repeat(1,len(tr[0]))
tr_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # train 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(tr))): # train 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(tr[i]))
    cols = tr[i]
    vals = np.repeat(1,len(tr[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    tr_csr = vstack([tr_csr, new_csr])


rows = np.repeat(0,len(te[0]))
cols = te[0]
vals = np.repeat(1,len(te[0]))
te_csr = csr_matrix((vals, (rows, cols)),[1, n_items]) # test(validation) 데이터의 첫 row를 csr_matrix화

for i in tqdm(range(1, len(te))): # test(validation) 데이터를 stack을 쌓는 방식으로 csr_matrix화
    rows = np.repeat(0,len(te[i]))
    cols = te[i]
    vals = np.repeat(1,len(te[i]))
    new_csr = csr_matrix((vals, (rows, cols)),[1, n_items])
    te_csr = vstack([te_csr, new_csr]) 

r = vstack([te_csr, tr_csr]).tocoo() # test, train 데이터를 병합하고 coo_matrix로 변환

## Logistic Matrix Factorization model for tags

In [16]:
model = LMF(factors = 50, regularization = 0.1, iterations = 40)
model.fit(r.T, show_progress = True)

tag_result = []

100%|██████████| 40/40 [00:40<00:00,  1.01s/it]


In [17]:
for u in tqdm(range(te_csr.shape[0])):
    tag_recommend = model.recommend(u, tr_csr, N = 200)
    tag_recommend = [x[0] for x in tag_recommend]
    tag_recommend = unique_item(tag_recommend, te[u])
    tag_recommend = [idx_to_iid[x] for x in tag_recommend][:10]
    tag_result.append(tag_recommend)

In [18]:
returnval = []
for _id, rec, tag_rec in tqdm(zip(te_ids, item_result_logit, tag_result)):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [19]:
import json
with open('result.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))